In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import pandas as pd

class FilteredDataset(ImageFolder):
    def __init__(self, root_dir, csv_path, transform=None, target_transform=None):
        super().__init__(root_dir, transform=transform, target_transform=target_transform)

        # Load excluded filenames from csv
        self.df = pd.read_csv(csv_path, header=None, names=['filename'])
        self.excluded_files = set(self.df['filename'].values.tolist())

        # Exclude images with filenames in the csv
        self.samples = [(path, target) for path, target in self.samples if path.split('/')[-1] not in self.excluded_files]


In [2]:
# Define the preprocessing transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(150),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
valid_transform = transforms.Compose([
    transforms.Resize(156),
    transforms.CenterCrop(150),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
train_dataset = FilteredDataset('data/train_set', 'data/exclude.csv', transform=train_transform)
valid_dataset = FilteredDataset('data/valid_set', 'data/exclude.csv', transform=valid_transform)

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=96, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=96, shuffle=True)

In [5]:
# Define the model architecture
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

/home/dnth/anaconda3/envs/fastdupv1/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dnth/anaconda3/envs/fastdupv1/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Train the model
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader)}")

Epoch 1 - Loss: 0.5508016788837861
Epoch 2 - Loss: 0.4412104717525495
Epoch 3 - Loss: 0.4000567503932382
Epoch 4 - Loss: 0.37917719294830243
Epoch 5 - Loss: 0.3817932825712931
Epoch 6 - Loss: 0.35811729232470196
Epoch 7 - Loss: 0.35321960998635715
Epoch 8 - Loss: 0.3363938052840784
Epoch 9 - Loss: 0.3341845804128517
Epoch 10 - Loss: 0.3258917426981894


In [8]:
# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in valid_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}")

Accuracy: 91.73333333333333
